<a href="https://colab.research.google.com/github/anniepeacock/DANSAR/blob/devel/burn_severity/GetData_Landsat_Ancillary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Remote Sensing of Wildfires: Landsat & Ancillary data for comparison with SAR**

*This notebook describes how to get the Landsat and relevant ancillary datasets over the Verdugo Mountains, Los Angeles for a comparison with SAR fire burn severity for the 2017 La Tuna Fire.*

Pre-processing steps to prepare data for this comparison/analysis

* Resample BAER & MTBS Landsat to UAVSAR extent & resolution
* Calculate UAVSAR-matching dNBR dates with Landsat
* Fire perimeter
* mountain extent
* vegetation type

## Running the Notebook
- This Notebook has both "text" and "code" cells. The text cells have text descriptions about running the notebooks and data interpretation.
- Code cells are a light gray and a "play" button appears in the upper left corner when your mouse is hovered over the cell.
- To run the content in the code cells, **select the play button** in the upper left corner of each code cell or **press shift-enter**.
- The figures and histograms are interactable via mouse/touch pad actions. Interactable functions can be selected on the top right corners of each figure.

This routine uses the following python libraries. Some are already included in the Google Colab environment and others are installed in the cell below before imported. Downloading new python packages to this environment may take a few seconds to complete.

<a name="s2"></a>
# 1. Downloading / Generating Files used in Analysis

1. BAER and MTBS Products originally downloaded from: https://burnseverity.cr.usgs.gov/viewer/?product=BAER
  * Fire ID: ca3429511836120170901
2. Calculated dNBR from UAVSAR-like dates with Landsat 8:
  * Script in GEE to generate dNBR with UAVSAR-like dates: https://code.earthengine.google.com/15eb2d2fa9e0a6649b145ce8e4c2ac30
    * dNBR geotiff is exported from GEE to Google Drive, then read in to this script to be resampled and subset to match the SAR datasets
  * Script Source: https://www.un-spider.org/advisory-support/recommended-practices/recommended-practice-burn-severity/burn-severity-earth-engine
3. Fire perimeter downloaded from the NIFC: https://data-nifc.opendata.arcgis.com/datasets/historic-perimeters-2017/explore
4. The geojson of the extent of the Verdugo Mountains was generated by tracing the extent of the mountains
5. CalVeg Download: https://www.fs.usda.gov/detail/r5/landmanagement/resourcemanagement/?cid=stelprdb5347192
  * https://data.fs.usda.gov/geodata/edw/datasets.php
  * Existing Vegetation: Region 5 - South Coast for Los Angeles
6. Processed UAVSAR and data stacks are staged here: https://uavsar.jpl.nasa.gov/cgi-bin/sar-notebooks.pl


In [2]:
!pip install rasterio --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 41.5 MB/s eta 0:00:00


In [3]:
import rasterio as rio
from rasterio.warp import reproject, Resampling
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import box, mapping

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Landsat Fire Severity Products

In [4]:
# Import dNBR for UAVSAR dates (2014 & 2017) 30 meter resolution
!wget https://uavsar.jpl.nasa.gov/SAR_NOTEBOOKS/Fire/dnbr_2014_2017.tif --quiet

# UAVSAR stack
!wget https://uavsar.jpl.nasa.gov/SAR_NOTEBOOKS/Fire/6m_stack.tif  --quiet

# NISAR Simulated stack
nisar_sim_path = ('/content/drive/MyDrive/UAVSAR_Science/LaTuna_Fire/GIS_Files/2024_data/nisarsim_stack.tif')
nisar_sim = rio.open(nisar_sim_path).read(1)

## Generate UAVSAR-resolution Landsat Fire Severity Products

In [6]:
# Open the target raster (6m_stack.tif) to extract its pixel size and extent
with rio.open("6m_stack.tif") as target_src:
    target_band = target_src.read(1)  # Choose the band you want to match
    target_profile = target_src.profile
    target_transform = target_src.transform
    target_crs = target_src.crs
    target_extent = target_src.bounds

    # Create a Polygon from the bounding box coordinates
    target_polygon = box(*target_extent)

# Open the source raster (dnbr_2014_2017.tif) to be resampled
with rio.open("dnbr_2014_2017.tif") as src:
    # Perform resampling to match the pixel size and extent of the target band
    resampled_data = src.read(
        out_shape=(target_band.shape[0], target_band.shape[1]),
        resampling=Resampling.nearest
    )

    # Define the transform for the resampled raster
    new_transform = target_transform

    # Prepare the resampled profile
    resampled_profile = src.profile.copy()
    resampled_profile.update({
        'width': target_band.shape[1],
        'height': target_band.shape[0],
        'transform': new_transform,
        'crs': target_crs
    })

    # Clip the resampled raster to the same extent as the target raster
    clipped_data, clipped_transform = mask(
        dataset=src,
        shapes=[mapping(target_polygon)],
        crop=True,
        filled=False
    )

    # Update the profile with the clipped transform and dimensions
    resampled_profile.update({
        'width': clipped_data.shape[2],
        'height': clipped_data.shape[1],
        'transform': clipped_transform
    })

    # Write the clipped and resampled raster to a new GeoTIFF file
    with rio.open("dnbr_sar_dates_resampled_uavsar.tif", 'w', **resampled_profile) as dst:
        dst.write(clipped_data)


## Add MTBS and BARC

## Generate NISAR Simulated Resolution Landsat Fire Severity Products

In [11]:
# Open the target raster (6m_stack.tif) to extract its pixel size and extent
with rio.open('/content/drive/MyDrive/UAVSAR_Science/LaTuna_Fire/GIS_Files/2024_data/nisarsim_stack.tif') as target_src:
    target_band = target_src.read(1)  # Choose the band you want to match
    target_profile = target_src.profile
    target_transform = target_src.transform
    target_crs = target_src.crs
    target_extent = target_src.bounds

    # Create a Polygon from the bounding box coordinates
    target_polygon = box(*target_extent)
    print(target_polygon)
# Open the source raster (dnbr_2014_2017.tif) to be resampled
with rio.open("dnbr_2014_2017.tif") as src:
    # Perform resampling to match the pixel size and extent of the target band
    resampled_data = src.read(
        out_shape=(target_band.shape[0], target_band.shape[1]),
        resampling=Resampling.nearest
    )

    # Define the transform for the resampled raster
    new_transform = target_transform

    # Prepare the resampled profile
    resampled_profile = src.profile.copy()
    resampled_profile.update({
        'width': target_band.shape[1],
        'height': target_band.shape[0],
        'transform': new_transform,
        'crs': target_crs
    })

    # Clip the resampled raster to the same extent as the target raster
    clipped_data, clipped_transform = mask(
        dataset=src,
        shapes=[mapping(target_polygon)],
        crop=True,
        filled=False
    )

    # Update the profile with the clipped transform and dimensions
    resampled_profile.update({
        'width': clipped_data.shape[2],
        'height': clipped_data.shape[1],
        'transform': clipped_transform
    })

    # Write the clipped and resampled raster to a new GeoTIFF file
    with rio.open("dnbr_sar_dates_resampled_nisarsim.tif", 'w', **resampled_profile) as dst:
        dst.write(clipped_data)


## Add MTBS and BARC

## Make Rasterized Data Stack of Fire Perimeter, Verdugo Mountains Perimeter

## CalVeg

In [ ]:
import geopandas as gpd

# Read GeoJSON file
gdf = gpd.read_file("calveg_latuna.geojson")

# Print dataframe header
print(gdf.head())

# Column - 'REGIONAL_DOMINACE_TYPE
